# Visualizing 3D data in napari
In this tutorial, you will learn:
- How to load images from hdf5 files
- How to open images in napari
- The difference between intensity images and label images
- How to visualize measurements on images using a napari plugin
- How to explore image data interactively

(Legend of stainings & meanings, from the last slide in Introduction_slides.pdf)
__Click-it and IF staining__  
- Pool embryos across timepoints in an Eppi (Eppendorf tube)  
- Click-it staining with __AF647-azide__ (following __5-EU__ (Ethynyl-uridine))  
    - stains __zygotic RNA__, thus zygotic DNA expression?  
    - on slide: 5-EU staining followed by TAMRA-azide for zygotic RNA staining.[[1]](https://www.thermofisher.com/order/catalog/product/E10345)  
- IF against β-Catenin (568) and PCNA (405)  
    - stained __bCatenin__ indicates location of cell membranes as it binds to cadherin.[[2]](https://en.wikipedia.org/wiki/Beta-catenin)[[3]](https://www.sciencedirect.com/topics/neuroscience/beta-catenin)  
    - stained Proliferating cell nuclear antigen (__PCNA__) indicated nuclei of proliferating cells (among others, DNA replication protein recruiter by forming a ring around DNA double helix at replication site (before fork)).[[4]](https://link.springer.com/chapter/10.1007/978-94-007-4572-8_15)  
- __SYTOX-488__ to stain __DNA__  
- _5-EU stains 5' untranslated region => zygotic RNA => zygotic gene expression. 
- 2 replicates  

__Questions__
- How is Click-iT staining done exactly, i.e. what do which chemicals do?
- Why does zygotic transcription appear to start at both 5'-ends of the zygotic DNA?
    - If transcription usually starts at both DNA ends, are there coding regions on both strands? (Which would make sense, I suppose. For some reason I did not consider this, recently.)
- (what does bcatenin stain?)
- (what does pcna stain?)

In [ ]:
import h5py
# We import some custom functions to handle the hdf5 files
from h5_files import h5_select, h5_summary, to_numpy

In [ ]:
# Get an overview of what's in an hdf5 file
file_path = r'bio325_material\20210416_Timecourse5EU_s5_NA.h5'
print(h5_summary(file_path))

filename:	20210416_Timecourse5EU_s5_NA.h5
condition:	ZE injected with 5-EU 2.5 - 5hpf, MeOH permeabilized, co-stained with SYTOX, bCatenin and PCNA

       name             shape   dtype     size     element_size_um  img_type    stain cycle wavelength level
    ch_00/0 (286, 2048, 2048)  uint16  2399.14 [1.0, 0.325, 0.325] intensity     5-EU     0        647     0
    ch_00/1 (286, 1024, 1024)  uint16   599.79   [1.0, 0.65, 0.65] intensity     5-EU     0        647     1
    ch_00/2   (286, 512, 512)  uint16   149.95     [1.0, 1.3, 1.3] intensity     5-EU     0        647     2
    ch_01/0 (286, 2048, 2048)  uint16  2399.14 [1.0, 0.325, 0.325] intensity bCatenin     0        568     0
    ch_01/1 (286, 1024, 1024)  uint16   599.79   [1.0, 0.65, 0.65] intensity bCatenin     0        568     1
    ch_01/2   (286, 512, 512)  uint16   149.95     [1.0, 1.3, 1.3] intensity bCatenin     0        568     2
    ch_02/0 (286, 2048, 2048)  uint16  2399.14 [1.0, 0.325, 0.325] intensity     PCNA   

In [ ]:
# Load in the images from the hd5f file
f = h5py.File(file_path, "r")
sytox = h5_select(f, {'stain': 'SYTOX', 'level': 0})[0]
sytox_np = to_numpy(sytox)

In [ ]:
# TODO: Check what we loaded with the h5_select function
print(sytox)
print(sytox.shape)  # numpy axis order of shape() = (z, x, y)
print(sytox_np[0:2])

<HDF5 dataset "0": shape (286, 2048, 2048), type "<u2">
(286, 2048, 2048)
[[[ 5  0  2 ...  2 10  7]
  [ 0  6  5 ...  0 10  5]
  [ 0  6  0 ...  3  9  0]
  ...
  [ 0  0  0 ... 19 16 21]
  [ 0  0 20 ...  0  9 14]
  [ 0  2 16 ...  0  0 12]]

 [[ 4  0  0 ...  7  0  8]
  [ 2  3  0 ...  6  6  0]
  [ 1  3  4 ...  5  3  5]
  ...
  [ 2  4  2 ...  9  0  2]
  [ 6 22 10 ...  4 14  0]
  [ 8 16  2 ... 12 12  2]]]


In [ ]:
# TODO: Load the PCNA, the bCatenin and the 5-EU images at level 0
print(h5_select(f, {'stain': 'PCNA', 'level': 0})[0])
PCNA = h5_select(f, {'stain': 'PCNA', 'level': 0})[0]
PCNA_np = to_numpy(PCNA)

bCatenin = h5_select(f, {'stain': 'bCatenin', 'level': 0})[0]
bCatenin_np = to_numpy(bCatenin)

zyg_5EU = h5_select(f, {'stain': '5-EU', 'level': 0})[0]
zyg_5EU_np = to_numpy(zyg_5EU)

<HDF5 dataset "0": shape (286, 2048, 2048), type "<u2">


In [ ]:
# Loading the segmentation mask for the nuclei
nuclei = h5_select(f, {'stain': 'nuclei', 'level': 2})[0]
    # stain: nuclei has shape (286, 512, 512), does/could this create problems with plotting (different dimensions)?
nuclei_np = to_numpy(nuclei)

In [ ]:
# TODO: Load the segmentation masks for the cells & the membranes

# loading segm. masks for cells
cells = h5_select(f, {'stain': 'cells', 'level': 2})[0]
cells_np = to_numpy(cells)

# loading segm. masks for membranes
membrane = h5_select(f, {'stain': 'membrane', 'level': 2})[0]
membrane_np = to_numpy(membrane)

In [ ]:
# Each dataset contains the actual data plus some metadata attributes
# As you can see here, the nuclear segmentation was done using cellpose: https://www.cellpose.org 
list(nuclei.attrs)

['cellpose_cellprob_th',
 'cellpose_d',
 'element_size_um',
 'img_type',
 'level',
 'stain']

## The napari viewer
napari is a fast, interactive, multi-dimensional image viewer for Python. It’s designed for browsing, annotating, and analyzing large multi-dimensional images.  
https://napari.org

In [ ]:
import napari

In [ ]:
# Open the image and the segmentation in the napari viewer
viewer = napari.view_image(sytox_np, scale=sytox.attrs['element_size_um'])
viewer.add_labels(nuclei_np, scale=nuclei.attrs['element_size_um'])
    # RuntimeError: Cannot run the event loop while another loop is running
    # solution: I already had napari opened (in a separate window) - anyhow

<Labels layer 'nuclei_np' at 0x2557615b520>

In [ ]:
# TODO: Add other intensity images & segmentations to the viewer & explore
viewer.add_image(PCNA_np, scale=PCNA.attrs['element_size_um'])
viewer.add_image(bCatenin_np, scale=bCatenin.attrs['element_size_um'])
viewer.add_image(zyg_5EU_np, scale=zyg_5EU.attrs['element_size_um'])
viewer.add_labels(cells_np, scale=cells.attrs['element_size_um'])
viewer.add_labels(membrane_np, scale=membrane.attrs['element_size_um'])
# Make sure to add the membrane segmentation to see the cell boundaries

<Labels layer 'membrane_np' at 0x25594fe04c0>

# Questions to look at during the exploration
_**Hint**: Before answering the first 3 questions here, move on to the 4th and 5th and to the next cell._
- How does nuclear size & cell size vary over the embryo?
  - label layer: nuclei or membranes, dataframe ...nuclei.csv or membranes.csv, feature: _see below_ (roundness)
  - biggest and roundest nuclei overlap mostly. physical size gives a clearer picture: ring formed area from top to bottom of blastula ('dome'), surrounding and surrounded laterally by smaller nuclei.
- How does the 5-EU signal vary over the different nuclei and what does that mean?
  - strongest signal at the very top and bottom layers. stronger signals from nuclei next to each other. Means that zygotic gene expression could be induced through contact with maternal cells (at top or bottom of blastula).
- How does PCNA vary over the embryo and what does that mean?
  - PCNA intensity overlaps very well with 5-EU intensity. Means that the most cell proliferation is happening where the cells started to express their own DNA.
- What would you need to measure to quantitatively answer those questions?
  - I would need some kind of visualisation method to show me the spatial distribution of the differing quantities asked about.
- What potential technical biases do you see in the images that we should be aware of when quantifying them?
  - The stainings must be applied evenly over the cells, otherwise intensity measurements are meaningless.
  - I think some other things are meant here, what?

## Visualize feature measurements
Using the napari feature visualization plugin _(**Hint**: in napari: click tab 'Plugins > napari-feature-visualization > feature_vis' to e.g. colourmap labels)_, look at some of the features, e.g.:  
- Physical Size of the nuclei or the cell. Roundness  
- EU intensity (mean, skewness)  
- PCNA intensities (mean)  

Do the quantifications help you answer the questions above?

__Prediction__: Sure they do - they help visualising exactly what I'm interested in, respectively what the questions are about. These quantifications do this by standardising a colourmap to the range of measurements in a certain label (e.g. EU intensity).

__Observation__: Yes, they do - I didn't use anything else to answer above questions.